In [32]:
from os import getenv

import pandas as pd
from simple_salesforce import Salesforce

In [33]:
SF_USER = getenv("SF_USER")
SF_PASS = getenv("SF_PASS")
SF_SECURITY_TOKEN = getenv("SF_SECURITY_TOKEN")

In [34]:
sf = Salesforce(username=SF_USER, password=SF_PASS, security_token=SF_SECURITY_TOKEN)

In [45]:
def soql_query_to_df(soql_query):
    soql_result = sf.query_all(soql_query)
    df = pd.DataFrame(soql_result.get("records"))
    df = df.drop(columns=["attributes"])
    return df

In [54]:
soql_query = """
    SELECT Id, Name, OwnerId, Country, Owner.Name
    FROM Lead
    """

In [55]:
df = soql_query_to_df(soql_query)

In [59]:
df.head()

,Id,Name,OwnerId,Country,Owner
0,00QDn000004au0VMAQ,James Butt,005Dn000004qKWyIAM,None,"{'attributes': {'type': 'Name', 'url': '/servi..."
1,00QDn000004au1oMAA,Yuki Whobrey,005Dn000004qKWyIAM,None,"{'attributes': {'type': 'Name', 'url': '/servi..."
2,00QDn000004au1pMAA,Fletcher Flosi,005Dn000004qKWyIAM,None,"{'attributes': {'type': 'Name', 'url': '/servi..."
3,00QDn000004au1qMAA,Bette Nicka,005Dn000004qKWyIAM,None,"{'attributes': {'type': 'Name', 'url': '/servi..."
4,00QDn000004au1rMAA,Veronika Inouye,005Dn000004qKWyIAM,None,"{'attributes': {'type': 'Name', 'url': '/servi..."


In [57]:
df['Owner'][0]

OrderedDict([('attributes',
              OrderedDict([('type', 'Name'),
                           ('url',
                            '/services/data/v52.0/sobjects/User/005Dn000004qKWyIAM')])),
             ('Name', 'Ops God')])

In [61]:
df['Owner'] = df['Owner'].apply(lambda x: x['Name'])

In [65]:
df.loc[df['Country'] == 'USA', 'OwnerId'] = '005Dn000004qKWyIAM'
df.loc[df['Country'] == 'USA', 'Owner'] = 'Ops God'

In [67]:
df.head()

,Id,Name,OwnerId,Country,Owner
0,00QDn000004au0VMAQ,James Butt,005Dn000004qKWyIAM,None,Ops God
1,00QDn000004au1oMAA,Yuki Whobrey,005Dn000004qKWyIAM,None,Ops God
2,00QDn000004au1pMAA,Fletcher Flosi,005Dn000004qKWyIAM,None,Ops God
3,00QDn000004au1qMAA,Bette Nicka,005Dn000004qKWyIAM,None,Ops God
4,00QDn000004au1rMAA,Veronika Inouye,005Dn000004qKWyIAM,None,Ops God


In [68]:
soql_query = """
    SELECT Id, Name
    FROM User
    """

user_df = soql_query_to_df(soql_query)
user_df.head()

,Id,Name
0,005Dn000005LenfIAC,Automated Process
1,005Dn000005LeniIAC,Data.com Clean
2,005Dn000004qKWyIAM,Ops God
3,005Dn000005LeneIAC,Integration User
4,005Dn000005LenkIAC,Security User


In [70]:
df.loc[df['Country'] != 'USA', 'OwnerId'] = '005Dn000004rqG8IAI'
df.loc[df['Country'] != 'USA', 'Owner'] = 'Salesforce Genius'

In [73]:
df = df.drop(columns=['Name','Country', 'Owner'])

In [76]:
list_of_records = df.to_dict("records")

In [79]:
sf.bulk.Lead.update(list_of_records)